In [2]:
import numpy as np
import pandas as pd
import re
import csv
import tldextract
from urllib.parse import urlparse
from src.urlfunctions import *

trimmed_file_path = 'trimmed_data.npy'

In [3]:
url_regex = "^(https?|ftp):\/\/([^\/]+)\/(([^\/?]+\/)*)?([^\/?]+\.\w+)?(\?([^=]+=[^&]+)(&([^=]+=[^&]+))*)?$"
url = "https://test.co.uk/?help=1"
match = re.match(url_regex, url)

def extract_urls_from_email(email_content):
    # Regex to match URLs
    url_pattern = r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"

    # Find all URLs in the email content
    urls = re.findall(url_pattern, email_content)
    return urls

def getNumSuspiciousKeywords(email_content):
    # need to somehow account for bypass word filter by using different letter
    text_lower = email_content.lower()
    keywords = ["urgent update", "pay now"]
    keyword_count = 0
    
    # Iterate through the keywords and count occurrences in the text
    for keyword in keywords:
        # Check if the keyword (case-insensitive) is present in the text
        if keyword.lower() in text_lower:
            keyword_count += 1
    
    return keyword_count

def getNumShortedUrls(email_content):
    short_domains = ['goo.gl', 'jmp.by']
    pattern = r'https?://(?:{})/\S+'.format('|'.join(re.escape(domain) for domain in short_domains))
    urls = re.findall(pattern, email_content)
    
    # Count the number of URLs found
    return len(urls)

def getNumIpAddresses(email_content):
    ipv4_regex = "\b(?:\d{1,3}\.){3}\d{1,3}\b"
    ipv6_regex = "(([0-9a-fA-F]{1,4}:){7,7}[0-9a-fA-F]{1,4}|([0-9a-fA-F]{1,4}:){1,7}:|([0-9a-fA-F]{1,4}:){1,6}:[0-9a-fA-F]{1,4}|([0-9a-fA-F]{1,4}:){1,5}(:[0-9a-fA-F]{1,4}){1,2}|([0-9a-fA-F]{1,4}:){1,4}(:[0-9a-fA-F]{1,4}){1,3}|([0-9a-fA-F]{1,4}:){1,3}(:[0-9a-fA-F]{1,4}){1,4}|([0-9a-fA-F]{1,4}:){1,2}(:[0-9a-fA-F]{1,4}){1,5}|[0-9a-fA-F]{1,4}:((:[0-9a-fA-F]{1,4}){1,6})|:((:[0-9a-fA-F]{1,4}){1,7}|:)|fe80:(:[0-9a-fA-F]{0,4}){0,4}%[0-9a-zA-Z]{1,}|::(ffff(:0{1,4}){0,1}:){0,1}((25[0-5]|(2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(25[0-5]|(2[0-4]|1{0,1}[0-9]){0,1}[0-9])|([0-9a-fA-F]{1,4}:){1,4}:((25[0-5]|(2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(25[0-5]|(2[0-4]|1{0,1}[0-9]){0,1}[0-9]))"
    
    ipv4_addresses = re.findall(ipv4_regex, email_content)
    ipv6_addresses = re.findall(ipv6_regex, email_content)
    
    return len(ipv4_addresses) + len(ipv6_addresses)

if match:
    protocol = match.group(1)
    domain = match.group(2)
    directory = match.group(3) if match.group(3) else ""  # Handle optional directory
    file = match.group(4) if match.group(4) else ""
    parameters = match.group(6) if match.group(6) else ""  # Handle optional parameters

    print("Protocol:", protocol)
    print("Domain:", domain)
    print("Directory:", directory)
    print("File:", file)
    print("Parameters:", parameters)



Protocol: https
Domain: test.co.uk
Directory: 
File: 
Parameters: ?help=1


In [4]:
csv.field_size_limit(100000000)
def read_csv_into_objects(file_path):
    rows = []
    with open(file_path, 'r') as csvfile:
        csv_reader = csv.reader(csvfile)
        header = next(csv_reader)  # Skip header
        print(header)
        for row in csv_reader:
            rows.append(row)
    return header, rows

email_file = "data/dataset_phishing.csv"
_, emails = read_csv_into_objects(email_file)
print(emails[0])
email_file = "data/Phishing_Email.csv"
_, emails = read_csv_into_objects(email_file)
print(emails[1])


['url', 'length_url', 'length_hostname', 'ip', 'nb_dots', 'nb_hyphens', 'nb_at', 'nb_qm', 'nb_and', 'nb_or', 'nb_eq', 'nb_underscore', 'nb_tilde', 'nb_percent', 'nb_slash', 'nb_star', 'nb_colon', 'nb_comma', 'nb_semicolumn', 'nb_dollar', 'nb_space', 'nb_www', 'nb_com', 'nb_dslash', 'http_in_path', 'https_token', 'ratio_digits_url', 'ratio_digits_host', 'punycode', 'port', 'tld_in_path', 'tld_in_subdomain', 'abnormal_subdomain', 'nb_subdomains', 'prefix_suffix', 'random_domain', 'shortening_service', 'path_extension', 'nb_redirection', 'nb_external_redirection', 'length_words_raw', 'char_repeat', 'shortest_words_raw', 'shortest_word_host', 'shortest_word_path', 'longest_words_raw', 'longest_word_host', 'longest_word_path', 'avg_words_raw', 'avg_word_host', 'avg_word_path', 'phish_hints', 'domain_in_brand', 'brand_in_subdomain', 'brand_in_path', 'suspecious_tld', 'statistical_report', 'nb_hyperlinks', 'ratio_intHyperlinks', 'ratio_extHyperlinks', 'ratio_nullHyperlinks', 'nb_extCSS', 'rat

In [12]:
getNumIpAddresses("http://shadetreetechnology.com/V4/validation/a111aedc8ae390eabcfa130e041a10a4")

0

In [4]:
site_file = "Phishing-Dataset/dataset_full.csv"
headers, ps = read_csv_into_objects(site_file)

def load_csv_to_numpy(filename):
    data = np.genfromtxt(filename, delimiter=',', skip_header=1)  # Skip header
    return data

csv_data = load_csv_to_numpy(site_file)
print(csv_data.shape)
print(csv_data.shape)
file_path = 'website_data.npy'
np.save(file_path, csv_data)

['qty_dot_url', 'qty_hyphen_url', 'qty_underline_url', 'qty_slash_url', 'qty_questionmark_url', 'qty_equal_url', 'qty_at_url', 'qty_and_url', 'qty_exclamation_url', 'qty_space_url', 'qty_tilde_url', 'qty_comma_url', 'qty_plus_url', 'qty_asterisk_url', 'qty_hashtag_url', 'qty_dollar_url', 'qty_percent_url', 'qty_tld_url', 'length_url', 'qty_dot_domain', 'qty_hyphen_domain', 'qty_underline_domain', 'qty_slash_domain', 'qty_questionmark_domain', 'qty_equal_domain', 'qty_at_domain', 'qty_and_domain', 'qty_exclamation_domain', 'qty_space_domain', 'qty_tilde_domain', 'qty_comma_domain', 'qty_plus_domain', 'qty_asterisk_domain', 'qty_hashtag_domain', 'qty_dollar_domain', 'qty_percent_domain', 'qty_vowels_domain', 'domain_length', 'domain_in_ip', 'server_client_domain', 'qty_dot_directory', 'qty_hyphen_directory', 'qty_underline_directory', 'qty_slash_directory', 'qty_questionmark_directory', 'qty_equal_directory', 'qty_at_directory', 'qty_and_directory', 'qty_exclamation_directory', 'qty_spac

In [40]:
trim_data = csv_data.transpose()
print(trim_data.shape)
a = np.delete(trim_data, np.s_[19:112:1], 0)
trim_headers = np.delete(headers, np.s_[19:112:1]).tolist()


print(a.shape)
print(type(a))
a=np.vstack([a,trim_data[96]])
a=np.vstack([a,trim_data[111]])
trim_headers.append(headers[96])
trim_headers.append(headers[111])
print(a.shape)
print(len(trim_headers))
print(trim_headers)

#np.save(trimmed_file_path, a.transpose())

(112, 88647)
(19, 88647)
<class 'numpy.ndarray'>
(21, 88647)
21
['qty_dot_url', 'qty_hyphen_url', 'qty_underline_url', 'qty_slash_url', 'qty_questionmark_url', 'qty_equal_url', 'qty_at_url', 'qty_and_url', 'qty_exclamation_url', 'qty_space_url', 'qty_tilde_url', 'qty_comma_url', 'qty_plus_url', 'qty_asterisk_url', 'qty_hashtag_url', 'qty_dollar_url', 'qty_percent_url', 'qty_tld_url', 'length_url', 'email_in_url', 'phishing']


In [26]:
trimmed_data = np.load(trimmed_file_path)
print(trimmed_data[3])
for cidx in range(trimmed_data.shape[1]):
    cdt = trimmed_data[:,cidx].dtype
    print(f"column {cidx}: {cdt}")

[ 4.  0.  2.  5.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
 81.  0.  1.]
column 0: float64
column 1: float64
column 2: float64
column 3: float64
column 4: float64
column 5: float64
column 6: float64
column 7: float64
column 8: float64
column 9: float64
column 10: float64
column 11: float64
column 12: float64
column 13: float64
column 14: float64
column 15: float64
column 16: float64
column 17: float64
column 18: float64
column 19: float64
column 20: float64


In [2]:

url = "https://www.google.com"
hostname, domain, path = get_domain(url)
extracted_domain = tldextract.extract(url)
domain = extracted_domain.domain+'.'+extracted_domain.suffix
subdomain = extracted_domain.subdomain
tmp = url[url.find(extracted_domain.suffix):len(url)]
pth = tmp.partition("/")
path = pth[1] + pth[2]
parsed = urlparse(url)


In [3]:
print(hostname, domain, path)

www.google.com google.com 


In [50]:
# transform csv into ML learnable data
df = pd.read_csv("data/selected_columns_1.csv")
#df = pd.read_csv("data/selected_columns_1.csv", sep=",", skipinitialspace=True)
# remove url
df = df.iloc[:, 1:]


# turn label into number
df['status'] = df['status'].map({'legitimate': 0, 'phishing': 1})
print(df.shape)

for column in df.columns:
    df[column] = pd.to_numeric(df[column], errors='coerce')

df=df.dropna()

#df['status'] = df['status'].astype(int)

float_col = df.select_dtypes(include=['float64'])
for col in float_col.columns.values:
    df[col] = df[col].astype('int64')

print(df.shape)
csv = df.to_csv("data/test.csv")


#df['status'] = df['status'].astype(int)

# URL WITH COMMA is causing trouble because pandas isn't respecting the double quote...... delete for now
#print(df.isna())

# convert to numpy
#scn = df.to_numpy()
scn = df.to_numpy(na_value=0)

scn[0] = scn[0].astype(int)
print(scn.shape)
#print(scn)

# convert to numpy
scn = df.to_numpy()
#for cidx in range(scn.shape[1]):
#    cdt = scn[:,cidx].dtype
#    print(f"column {cidx}: {cdt}")

print(scn.shape)
print(scn[0])
print(scn[0].shape)

(11184, 46)
(11161, 46)
(11161, 46)
(11161, 46)
[37 19  3  0  0  0  0  0  0  0  0  0  3  0  1  0  0  0  0  1  0  0  0  1
  0  0  0  0  0  3  0  0  4  0  0  0  0  0  0 45 -1  0  1  1  4  0]
(46,)


In [48]:
scn_filename = 'website_data_more.npy'
np.save(scn_filename, scn)

np.random.shuffle(scn)
split_ratio = 0.8
si = int(split_ratio * len(scn))
train_data = scn[:si]
test_data = scn[si:]
print(train_data.shape)
print(test_data.shape)

np.save("2train.npy", train_data)
np.save("2test.npy", test_data)

(8928, 46)
(2233, 46)
